# BeautifulSoup Scraping

In [77]:
# import the libraries
import requests
import csv
from bs4 import BeautifulSoup

In [95]:
# convert model names to brands
import csv

# open the models file and use that to replace model names with brand names. 
# format of file is brand, model
file = open('models.csv', 'r')
csv_file = csv.reader(file)

# don't store brand names that are weird from the csv file. like 'problems' and 'car' those arent brand names
weird_brands = ['car', 'hyndai kia', 'hyundai,', 'hyundai.', 'kia,', 'kia.', 
                'mercedes benz', 'mercedes-benz', 'nissan.', 'problem', 'seat', 'toyata','volkwagen' ]

# store the brand names
brands = []
distinct_brands = []
for i in csv_file:
    if(i[0] in weird_brands):
        continue
    else:
        brands.append(i)
        if(i[0] not in distinct_brands):
            distinct_brands.append(i[0])
#     parsedData = parsedData.replace(i[1], i[0])

print('Finished storing this csv file into a list')
# close the file
file.close()

Finished storing this csv file into a list


In [82]:
brands

[['acura', 'integra'],
 ['acura', 'Legend\xa0'],
 ['acura', 'vigor'],
 ['acura', 'rlx'],
 ['acura', 'ILX'],
 ['acura', 'MDX'],
 ['acura', 'RDX'],
 ['acura', 'TSX'],
 ['acura', 'RSX'],
 ['audi', 'quattro'],
 ['audi', 'a4'],
 ['audi', 'a6'],
 ['audi', 'a5'],
 ['audi', 'a7'],
 ['audi', 'q3'],
 ['audi', 'q5'],
 ['audi', 'q7'],
 ['audi', 'a8'],
 ['audi', 'a3'],
 ['bmw', '3-series'],
 ['bmw', '5-series'],
 ['bmw', '6-series'],
 ['bmw', '4-series'],
 ['bmw', 'xdrive'],
 ['bmw', 'alpina'],
 ['bmw', '330ci'],
 ['bmw', '328i'],
 ['bmw', '330c'],
 ['bmw', '335i'],
 ['bmw', '335d'],
 ['bmw', '528i'],
 ['bmw', '535i'],
 ['bmw', '128i'],
 ['bmw', '640i'],
 ['bmw', '3er'],
 ['bmw', '5er'],
 ['bmw', 'M5'],
 ['bmw', 'M3'],
 ['bmw', 'M6'],
 ['bmw', 'bmws'],
 ['bmw', 'bmwesque'],
 ['buick', 'park avenue'],
 ['buick', 'la crosse'],
 ['buick', 'lacrosse'],
 ['buick', 'century'],
 ['buick', 'enclave'],
 ['buick', 'encore'],
 ['buick', 'regal'],
 ['buick', 'buicks'],
 ['cadillac', 'escalade'],
 ['cadillac', 

## Scraping all pages

In [79]:
# we'll be using the cars and conversations forum
url = 'https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations'

In [94]:
# number of forum pages
num_pages = 2575

In [96]:
# store the comments in a string
all_comments = ""

# store output: date, userid, message
output_csv = []

# go through each page
for page_number in range(1, num_pages + 1):
    # first page string of url isn't anything special
    try:
        if(page_number == 1):
            new_url = url
        else:
            # end url format is /p220 etc.
            new_url = url + "/p" + str(page_number)
            
        print(new_url)
        # collect the url with requests library
        page = requests.get(new_url)
        
        
        # get the html of the page in string form
        page_html = page.text
        
        # create the BeautifulSoup object that takes in the html in str form and a html/xml parser of choice either html.parser or lxml
        soup = BeautifulSoup(page_html, 'html.parser')
        
        # inspect the elements from the webpage to know where stuff is
        # in this case the comments are under a <p> tag which are all under a <div class = "Message userContent"</div/ tag 
        # which are all under a <div class = "MessageList DataList Comments"> tag
    
        # Pull all text from the 'MessageList DataList Comments' divs
        messagelist = soup.find(class_ = "MessageList DataList Comments")
        
        # within the 'MessageList DataList Comments' divs pull all text from 'Message userContent' divs
        #usermessages = messagelist.find_all(class_ =  "Message userContent")
        usermessages = messagelist.find_all(class_ = "Comment")
        
        # now extract just the <p> tags from all comments! 
        for i in range(len(usermessages)):
            # let's grab the date and username of the post which is under class = "Item-Header CommentHeader"
            # in a <time> tag and class = "Username"
            
            commentheader = usermessages[i].find(class_ = 'Item-Header CommentHeader')
            
            # grab the date. Format: June 18, 2014  3:53PM is stored in title = ' ' of <time  >
            date_of_post = commentheader.find("time", title = True)
            
            # variable to store date
            date_of_post = date_of_post['title']
            
            # grab the username
            username = commentheader.find(class_ = "Username")
            # variable to store username
            username = username.string
            
            # now grab text under Message userContent
            messageheader = usermessages[i].find(class_ = 'Message userContent')
            
            # a user comment might have multiple <p> tags
            # [s.extract() for s in usermessages[i](class_ = 'UserQuote')]
            [s.extract() for s in messageheader(class_ = 'UserQuote')]
            [s.extract() for s in messageheader('a')]
            [s.extract() for s in messageheader('img')]
            [s.extract() for s in messageheader('br')]
            [s.extract() for s in messageheader('blockquote')]
            
            
            
            # variable to store user message
            message = messageheader.prettify()
            
            
                
            # let's do some string manipulation
            # remove the newline characters, '<p>', '</p','said:'
            message = message.replace('\n', '')
            message = message.replace(r"<p>", "")
            message = message.replace(r"</p>", "")
            message = message.replace(r"</p>", "")
            message = message.replace(r"said:", "")
            message = message.replace(r":", "")
            message = message.replace(r'</div>' , "")
            message = message.lower()
            message = message.replace(r'<div class="message usercontent">' , "")
            message = message.replace(r'&gt;' , "")
            
            
            # deal with brand name
            for k in brands:
                message = message.replace(k[1], k[0])
            
            # add to all_comments
            all_comments += message
            
            # create a tuple with Date, Userid, Message and store into output_csv
            row = date_of_post, username, message
            output_csv.append(row)
            
            # print this if you want to see some output
            # print('username: ' + str(username))
            # print('date: ' + str(date_of_post))
            # print('message: ' + message)
            
        # print some checks
        print('Page ' + str(page_number) + ' complete')
    except:
        print('page ' + str(page_number) + ' failed')
        continue

https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations
Page 1 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2
Page 2 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p3
Page 3 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p4
Page 4 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p5
Page 5 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p6
Page 6 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p7
Page 7 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p8
Page 8 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p9
Page 9 complete
https://forums.edmunds.com/disc

Page 75 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p76
Page 76 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p77
Page 77 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p78
Page 78 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p79
Page 79 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p80
Page 80 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p81
Page 81 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p82
Page 82 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p83
Page 83 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p84
Page 84 co

Page 149 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p150
Page 150 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p151
Page 151 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p152
Page 152 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p153
Page 153 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p154
Page 154 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p155
Page 155 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p156
Page 156 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p157
Page 157 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversatio

Page 223 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p224
Page 224 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p225
Page 225 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p226
Page 226 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p227
Page 227 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p228
Page 228 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p229
Page 229 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p230
Page 230 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p231
Page 231 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversatio

Page 297 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p298
Page 298 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p299
Page 299 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p300
Page 300 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p301
Page 301 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p302
Page 302 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p303
Page 303 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p304
Page 304 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p305
Page 305 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversatio

...

Page 2260 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2261
Page 2261 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2262
Page 2262 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2263
Page 2263 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2264
Page 2264 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2265
Page 2265 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2266
Page 2266 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2267
Page 2267 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2268
Page 2268 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members

Page 2332 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2333
Page 2333 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2334
Page 2334 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2335
Page 2335 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2336
Page 2336 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2337
Page 2337 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2338
Page 2338 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2339
Page 2339 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2340
Page 2340 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members

Page 2404 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2405
Page 2405 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2406
Page 2406 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2407
Page 2407 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2408
Page 2408 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2409
Page 2409 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2410
Page 2410 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2411
Page 2411 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2412
Page 2412 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members

Page 2476 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2477
Page 2477 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2478
Page 2478 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2479
Page 2479 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2480
Page 2480 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2481
Page 2481 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2482
Page 2482 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2483
Page 2483 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2484
Page 2484 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members

Page 2548 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2549
Page 2549 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2550
Page 2550 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2551
Page 2551 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2552
Page 2552 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2553
Page 2553 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2554
Page 2554 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2555
Page 2555 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2556
Page 2556 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members

In [97]:
output_csv[-1]

('October  6, 2018 12:55AM',
 'henryn',
 " a few years back, before i returned to work, i played around quite a bit with dvd movies.  as long as it's not copy protected, it is fairly easy to copy to your hard drive, and then change the region coding.  if it's copy protected, that is a different story. and even if you do remove the region coding, you still have the problem of ntsc vs pal.  your dvd player and tv may handle pal -- and then again, maybe not.")

## Throw output_csv list into a csv file and place all_text into a text file

In [98]:
# output into csv file
# throw this into a text file just in case
with open('finalcomments.csv','w', encoding = 'utf-8', newline = '') as out:
    csv_out = csv.writer(out)
    csv_out.writerow (['date','username', 'comment'])
    for row in output_csv:
        csv_out.writerow(row)

print('write csv is complete')

write csv is complete


In [113]:
# sanity check
import pandas as pd
pd.read_csv('finalcomments.csv')

,date,username,comment
0,"June 18, 2014 3:53PM",graphicguy,i'm 2nd post....yee haw!!!!!!!! do i get...
1,"June 18, 2014 4:01PM",bwia,"great, but will it be possible to refer to ..."
2,"June 18, 2014 4:08PM",imidazol97,"if you have posted in that discussion, clic..."
3,"June 18, 2014 4:21PM",stickguy,"if you want to bookmark this one, there is ..."
4,"June 18, 2014 4:43PM",bwia,"thanks , it worked like a charm."
5,"June 18, 2014 4:49PM",bwia,thanks imidazol97. i will try that later bu...
6,"June 18, 2014 4:51PM",Kirstie_H,"no, but it's almost like you're famous. ju..."
7,"June 18, 2014 5:35PM",cdnpinhead,"okay, here am i also. now i need to add th..."
8,"June 18, 2014 5:39PM",graphicguy,wow! i'm almost famous....... thanks......
9,"June 18, 2014 5:50PM",ab348,"wow, that's a throwback to the old, old, ol..."


In [99]:
# throw this into a text file just in case
new_file = open('alltext.txt', 'w', encoding = 'utf-8')
new_file.write(all_comments)
new_file.close()

## Word Frequency Check

In [100]:
# grab word frequency using nltk library
import nltk
# stop words
from nltk.corpus import stopwords
nltk.download()
nltk.download('stopwords')

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [101]:
# tokenize the text 
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(all_comments)

In [110]:
# let's do a word count
from collections import Counter

# counter object
words = Counter()

# update counter with new words
words.update(tokens)

In [111]:
# check
words.most_common()[:10]

[('the', 247073),
 ('i', 161752),
 ('a', 144818),
 ('to', 133667),
 ('and', 113101),
 ('it', 92318),
 ('of', 82035),
 ('that', 75989),
 ('in', 75401),
 ('is', 61736)]

So these are the most common words, but they are also the same in any english text. Therefore they are not very insightful. Hence, let's remove single characters, numbers, and common words.

In [112]:
# store the stop words
stopwords = stopwords.words('english')[:]

AttributeError: 'list' object has no attribute 'words'

In [105]:
# lets do this again: remove stop words, single letter variable names
words = Counter(x for x in tokens if x not in stopwords and x.isdigit() == False and len(x) != 1)

In [106]:
# check
words.most_common()[:10]

[('car', 29770),
 ('one', 21534),
 ('like', 21158),
 ('would', 19675),
 ('get', 17488),
 ('new', 12271),
 ('think', 12216),
 ('good', 11715),
 ('time', 11441),
 ('back', 10375),
 ('cars', 10267),
 ('know', 9632),
 ('years', 9343),
 ('could', 8754),
 ('much', 8633),
 ('even', 8379),
 ('go', 8368),
 ('got', 8247),
 ('people', 8070),
 ('really', 7825),
 ('see', 7722),
 ('well', 7692),
 ('going', 7517),
 ('way', 7365),
 ('said', 7129),
 ('want', 6911),
 ('year', 6869),
 ('still', 6757),
 ('right', 6528),
 ('make', 6374),
 ('span', 6213),
 ('lot', 5897),
 ('dealer', 5891),
 ('also', 5746),
 ('never', 5742),
 ('old', 5657),
 ('better', 5611),
 ('take', 5584),
 ('sure', 5526),
 ('drive', 5460),
 ('used', 5322),
 ('buy', 5312),
 ('first', 5271),
 ('something', 5268),
 ('last', 5251),
 ('many', 5158),
 ('ford', 5068),
 ('probably', 5062),
 ('around', 5026),
 ('day', 4933),
 ('miles', 4880),
 ('us', 4834),
 ('nice', 4760),
 ('work', 4691),
 ('class', 4666),
 ('price', 4661),
 ('say', 4637),
 ('n

In [107]:
# now let's do this again but with only brand names
words = Counter(x for x in tokens if x not in stopwords and x.isdigit() == False and len(x) != 1 and x in distinct_brands)

In [108]:
# check
words.most_common()[:]

[('ford', 5068),
 ('bmw', 3271),
 ('chevrolet', 3169),
 ('volkswagen', 2936),
 ('honda', 2716),
 ('mercedes', 2539),
 ('toyota', 2467),
 ('cadillac', 2437),
 ('audi', 2413),
 ('buick', 1886),
 ('nissan', 1662),
 ('chrysler', 1267),
 ('dodge', 1140),
 ('sedan', 1025),
 ('kia', 1005),
 ('subaru', 931),
 ('hyundai', 923),
 ('lincoln', 913),
 ('mazda', 705),
 ('acura', 673),
 ('volvo', 583),
 ('pontiac', 387),
 ('infiniti', 380),
 ('mitsubishi', 292),
 ('mercury', 204),
 ('saturn', 67),
 ('suzuki', 34)]